In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import uuid
import json
import os
import re
import sys
from tqdm import tqdm
from tika import parser
import markdown
from pathlib import Path
from html.parser import HTMLParser

In [2]:
worker_pages = ['הודעה_מוקדמת_לפיטורים','מנוחה_שבועית','שוויון_בעבודה','הפסקות_בעבודה',
                'יום_עבודה_ושבוע_עבודה','זמן_תפילה_בעבודה',
                'ערעור_על_פיטורים','העסקת_נשים_במשמרת_לילה','מענק_שחרור_לחיילים_משוחררים_ומסיימי_שירות_לאומי-אזרחי',
                'דמי_אבטלה_לחיילים_משוחררים_ומסיימי_שירות_לאומי-אזרחי','גמול_עבור_שעות_נוספות',
                'שכר_שווה','תשלום_דמי_חופשה','איסור_העסקת_נוער_בעבודת_לילה','איסור_התעמרות_בעבודה','מניעת_עישון_במקום_העבודה','חופשה_שנתית',
                'פיצויי_פיטורים_לעובד_שמעסיקו_נפטר',]

pension_pages = ['גיל_פרישה_מעבודה','קרן_פנסיה','איסור_פיטורים_בגיל_פרישה_של_אלמנות_צה"ל_והורים_שכולים_שילדם_נפטר_עקב_שירות_צבאי',
                 'ביטוח_מנהלים','קופת_גמל_להשקעה','חובת_ביטוח_פנסיוני_לעובדים','קרן_השתלמות',
                 'ביטוח_פנסיוני_לעובד_עצמאי','זכותון_קצבת_זיקנה','השכר_הממוצע','כיסוי_ביטוחי_במסגרת_הביטוח_הפנסיוני']

In [3]:
context_df=pd.DataFrame(columns=["title","paragraphs"])

In [4]:
for title in worker_pages+pension_pages:
    paragraphs=[]
    url=f"https://www.kolzchut.org.il/he/{title}"
    page=requests.get(url)
    soup = BeautifulSoup(page.content,'html.parser')
    for tag in soup.find_all('ul'):
        if(len(tag.text)>500 and '.' in tag.text):
            paragraphs.append(tag.text)
    context_df.loc[len(context_df.index)] = [title, paragraphs]
    print(context_df.shape)

(1, 2)
(2, 2)
(3, 2)
(4, 2)
(5, 2)
(6, 2)
(7, 2)
(8, 2)
(9, 2)
(10, 2)
(11, 2)
(12, 2)
(13, 2)
(14, 2)
(15, 2)
(16, 2)
(17, 2)
(18, 2)
(19, 2)
(20, 2)
(21, 2)
(22, 2)
(23, 2)
(24, 2)
(25, 2)
(26, 2)
(27, 2)
(28, 2)
(29, 2)


In [5]:
def df2squadFormat(df, output_dir=None, filename=None):
    """
     df : pandas.DataFrame
         a pandas dataframe with columns ['title', 'paragraphs']

     output_dir : str, optional
         Enable export of output (the default is None)
     filename : str, optional
         [description]
    """

    json_data = {}
    json_data["version"] = "v1.1" # required for SQuAD v1.1
    # v1.1 needs only answer questions when possible and not determine when no answer is supported by the paragraph.
    json_data["data"] = []

    for idx, row in tqdm(df.iterrows()):
        temp = {"title": row["title"], "paragraphs": []}
        for paragraph in row["paragraphs"]:
            temp["paragraphs"].append({"context": paragraph, "qas": []})
        json_data["data"].append(temp)

    if output_dir:
        with open(os.path.join(output_dir, "{}.json".format(filename)), "w") as outfile:
            json.dump(json_data, outfile)

    return json_data

In [6]:
json_data = df2squadFormat(df=context_df, output_dir='.', filename='rights-dataset')

29it [00:00, 9645.14it/s]


In [7]:
json_data

{'version': 'v1.1',
 'data': [{'title': 'הודעה_מוקדמת_לפיטורים',
   'paragraphs': [{'context': 'יחסי העובד-מעסיק בין הצדדים מסתיימים עם תום תקופת ההודעה המוקדמת (כלומר, בסיום יחסי העבודה בפועל).\nבמהלך התקופה שמרגע מתן הודעת הפיטורים ועד סיום העבודה, המעסיק חייב להעניק לעובד את כל תנאי העבודה שהיה זכאי להם לפני כן.\nהעובד מחויב במהלך אותה תקופה להעמיד את עצמו לרשות העבודה ולהמשיך לעבוד באותו היקף ובאותה רצינות, כפי שנדרשו ממנו לפני כן.\nהחובה לתת לעובד הודעה מוקדמת  חלה גם בעת פיטורים עקב חילופי מעסיקים. חובה זו חלה  גם אם העובד ממשיך לעבוד באותו מקום עבודה אצל המעסיק החדש. לקריאת פסק דין בסוגיה זו ראו בקישור הזה.',
     'qas': []},
    {'context': 'על פי סעיף 5 לחוק חופשה שנתית, ימי הודעה מוקדמת לפיטורים לא יבואו במניין ימי החופשה אלא אם עלו על 14 ימים. כלומר, 14 ימים מתוך תקופת ההודעה המוקדמת לא יחפפו את תקופת החופשה ויש להבטיח כי לפחות 14 ימים מתוך תקופת ההודעה המוקדמת לא ינוצלו לחופשה.\nעל פי  פסיקת בית הדין הארצי לעבודה, כאשר המעסיק מוותר  על עבודתו של העובד בתקופת ההודעה המוקדמת,